In [8]:
import numpy as np

from assignment_util import *
from SAMinimizer import *
from TSP import * 
from cooling import *

In [15]:
adj_matrices = load_all_adj_matrices()
# seethe
assert  list(
            sorted(
                map(
                    lambda arr: arr.shape,
                    adj_matrices))) == [(51, 51), (280, 280), (442, 442)]

for amat in adj_matrices:
    for row in amat:
        for elmt in row:
            assert elmt >= 0, f"{elmt}"

In [10]:
x = np.array([[1,2,3],[4,5,6],[-5,2,5]])
np.all(x) > 0

True